In [1]:
import pandas as pd
import torch
from torch import nn
import jieba
from gensim.models import Word2Vec
import numpy as np

import Ipynb_importer

from data_pre import load_tsv
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

评论数目（总体）：119988
评论数目（正向）：59993
评论数目（负向）：59995
训练集大小: 95990
测试集大小: 23998
训练集和测试集已保存到 CSV 文件中。
Successfully


In [2]:
# 数据读取
def load_txt(path):
    with open(path,'r',encoding='utf-8') as f:
        data=[[line.strip()] for line in f.readlines()]
        return data
 
train_x=load_txt('train.txt')
test_x=load_txt('test.txt')
train=train_x+test_x
X_all=[i for x in train for i in x]
 
_, train_y = load_tsv("./data/train_dataset.csv")
_, test_y = load_tsv("./data/test_dataset.csv")

In [3]:
# 训练Word2Vec模型
word2vec_model = Word2Vec(sentences=X_all, vector_size=100, window=5, min_count=1, workers=4)

In [4]:
# 将文本转换为Word2Vec向量表示
def text_to_vector(text):
    vector = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv]
    return sum(vector) / len(vector) if vector else [0] * word2vec_model.vector_size
 
X_train_w2v = [[text_to_vector(text)] for line in train_x for text in line]
X_test_w2v = [[text_to_vector(text)] for line in test_x for text in line]

In [5]:
# 将词向量转换为PyTorch张量
X_train_array = np.array(X_train_w2v, dtype=np.float32)
X_train_tensor = torch.Tensor(X_train_array)
X_test_array = np.array(X_test_w2v, dtype=np.float32)
X_test_tensor = torch.Tensor(X_test_array)

In [6]:
# 确保 train_y 和 test_y 是数值类型的列表
train_y = train_y.values  # 如果是 DataFrame 或 Series
test_y = test_y.values  # 如果是 DataFrame 或 Series

# 如果需要，可以进一步确保它们是整数类型
train_y = train_y.astype(int)
test_y = test_y.astype(int)

In [7]:
#使用DataLoader打包文件
train_dataset = TensorDataset(X_train_tensor, torch.LongTensor(train_y))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(X_test_tensor,torch.LongTensor(test_y))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [8]:
# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
 
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # 取序列的最后一个输出
        return output

# 定义模型
input_size = word2vec_model.vector_size
hidden_size = 50  # 你可以根据需要调整隐藏层大小
output_size = 2  # 输出的大小，根据你的任务而定
 
model = LSTMModel(input_size, hidden_size, output_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

In [9]:
if __name__ == "__main__":
    # 训练模型
    num_epochs = 10
    log_interval = 100  # 每隔100个批次输出一次日志
    loss_min=100
    train_losses = []
    
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
             # 确保输入数据和目标张量在同一设备上
            data = data.to(device)
            target = target.to(device)
            
             # 确保目标张量是一维的
            if target.dim() == 2:
                target = target.squeeze()
            outputs = model(data)
            loss = criterion(outputs, target)
 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
 
            if batch_idx % log_interval == 0:
                print('Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}'.format(
                    epoch + 1, num_epochs, batch_idx, len(train_loader), loss.item()))

            # 记录训练损失
            train_losses.append(loss.item())
            
            # 保存最佳模型
            if loss.item()<loss_min:
                loss_min=loss.item()
                torch.save(model, 'model.pth')
 
    # 模型评估
    with torch.no_grad():
        model.eval()
        correct = 0
        total = 0
        all_targets = []
        all_predictions = []
        
        for data, target in test_loader:
            # 确保输入数据和目标张量在同一设备上
            data = data.to(device)
            target = target.to(device)

             # 确保目标张量是一维的
            if target.dim() == 2:
                target = target.squeeze()
            
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

         # 收集所有目标和预测值
            all_targets.extend(target.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
 
        accuracy = correct / total
        print('Test Accuracy: {:.2%}'.format(accuracy))

        # 计算混淆矩阵
        cm = confusion_matrix(all_targets, all_predictions)
        print("混淆矩阵：")
        print(cm)

        # # 绘制混淆矩阵
        # plt.figure(figsize=(8, 6))
        # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
        #             xticklabels=['Negative', 'Positive'], 
        #             yticklabels=['Negative', 'Positive'])
        # plt.xlabel('Predicted')
        # plt.ylabel('Actual')
        # plt.title('Confusion Matrix')
        # plt.savefig('./confusion_matrix.png')
        # plt.show()

    # # 绘制训练过程中的损失值趋势
    # plt.figure(figsize=(10, 6))
    # plt.plot(train_losses, label='Training Loss')
    # plt.xlabel('Batch Index')
    # plt.ylabel('Loss')
    # plt.title('Training Loss Trend')
    # plt.legend()
    # plt.savefig('training_loss_trend.png')  # 保存训练损失趋势图像
    # plt.show()

Epoch [1/10], Batch [0/1500], Loss: 0.7286
Epoch [1/10], Batch [100/1500], Loss: 0.6419
Epoch [1/10], Batch [200/1500], Loss: 0.5759
Epoch [1/10], Batch [300/1500], Loss: 0.4484
Epoch [1/10], Batch [400/1500], Loss: 0.3757
Epoch [1/10], Batch [500/1500], Loss: 0.3201
Epoch [1/10], Batch [600/1500], Loss: 0.3599
Epoch [1/10], Batch [700/1500], Loss: 0.3347
Epoch [1/10], Batch [800/1500], Loss: 0.3051
Epoch [1/10], Batch [900/1500], Loss: 0.2565
Epoch [1/10], Batch [1000/1500], Loss: 0.3670
Epoch [1/10], Batch [1100/1500], Loss: 0.2532
Epoch [1/10], Batch [1200/1500], Loss: 0.2865
Epoch [1/10], Batch [1300/1500], Loss: 0.3909
Epoch [1/10], Batch [1400/1500], Loss: 0.2128
Epoch [2/10], Batch [0/1500], Loss: 0.1888
Epoch [2/10], Batch [100/1500], Loss: 0.4227
Epoch [2/10], Batch [200/1500], Loss: 0.2787
Epoch [2/10], Batch [300/1500], Loss: 0.2739
Epoch [2/10], Batch [400/1500], Loss: 0.2470
Epoch [2/10], Batch [500/1500], Loss: 0.2653
Epoch [2/10], Batch [600/1500], Loss: 0.3461
Epoch [2/